# Notebook to play with ATRs

Latest version: 2024-08-23  
Author: MvS

## Description

Notebook to illustrate and calculate the average true range (ATR) indicator:

- measure volatility of equity prices, including gaps between trading periods


## Result

It has multiple [uses](https://skilltrader.de/average-true-range-strategien/):

- dynamic stop-loss calculation,
- breakout signal,
- pullback strategy.


In [ ]:
import yfinance as yf
import datetime
from math import log2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Standard SMAs
periods = [200, 50]

# stock = "FROG"
# stock = "TSLA"
# stock = "NVDA"
# stock = "SAP.TO"
# stock = "GLEN.L"
stock = "ADM.L"
# stock = "SPX.L"
# stock = "JPM"

dt_end = datetime.datetime.today()
# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - this requires double the number of points to create the averaging
#  - plus considering non-trading days - yfinance returns only trading days, howevers
dt_data_start = dt_end - datetime.timedelta(days=max(periods) * 3)

try:
    # Grab sufficient stock data for averaging SMAs
    load_df = yf.download(
        f"{stock}",
        start=dt_data_start.strftime('%Y-%m-%d'),
        end=dt_end.strftime('%Y-%m-%d'),
        progress=False,
    )

    assert load_df.shape[1] == 6 and load_df.shape[0] > max(periods)

except AssertionError:
    print(f"Download failed for symbol {stock}.  Skipping...")

### Definiton and calculation of ATR indicator

Source: [Investopedia](https://www.investopedia.com/terms/a/atr.asp)

1. True range is defined as `TR = Max[(H−L), ∣H−Cp​∣, ∣L−Cp​∣]` where:

    - `H`: Today’s high
    - `L`: Today’s low
    - `Cp`: Yesterday’s closing price
    - `Max`: Highest value of the three terms

2. ATR is the arithmetic mean of the daily true ranges over previous periods

3. Added a low-end volatility estimate which I define as `LR = Max[((Cp+H)/2.0) - L, (Cp - L)]`


In [ ]:
# Now that we calculated the SMAs and EMAs, we can remove the data points before the actual AOI that we want.
stock_df = load_df[-max(periods) :].copy()


# calculate true range
def true_range(x):
    high = x['High']
    low = x['Low']
    close_p = x['ClosePrev']

    if pd.isna(close_p):
        return high - low
    else:
        return max(
            high - low,
            abs(high - close_p),
            abs(close_p - low),
        )


stock_df['ClosePrev'] = stock_df['Close'].shift(1)


# calculate low range
def low_range(x):
    open = x['Open']
    high = x['High']
    low = x['Low']
    close_p = x['ClosePrev']

    if pd.isna(close_p):
        return (open + high) / 2.0 - low
    else:
        return max((close_p + high) / 2.0 - low, close_p - low, 0)


# calculate high range
def high_range(x):
    open = x['Open']
    high = x['High']
    low = x['Low']
    close_p = x['ClosePrev']

    if pd.isna(close_p):
        return high - (open + low) / 2.0
    else:
        return max(high - (close_p + low) / 2.0, high - close_p, 0)


stock_df['ClosePrev'] = stock_df['Close'].shift(1)

stock_df['TR'] = stock_df.apply(true_range, axis=1)
stock_df['LR'] = stock_df.apply(low_range, axis=1)
stock_df['HR'] = stock_df.apply(high_range, axis=1)

# Define the columns and their styles
columns = [
    'High',
    'Low',
    'Close',
    'ClosePrev',
    'TR',
]
colors = ['darkgreen', 'red', 'forestgreen', 'orangered', 'grey']
linestyles = ['-', '-', ':', ':', '-']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = stock_df[col].plot(color=color, linestyle=style, ax=ax)


# clean up
stock_df.drop(
    ['ClosePrev'],
    axis=1,
    inplace=True,
    errors='ignore',
)

### Plotting the Chart

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with secondary y-axis enabled
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the candlestick chart
fig.add_trace(
    go.Candlestick(
        x=stock_df.index,
        open=stock_df['Open'],
        high=stock_df['High'],
        low=stock_df['Low'],
        close=stock_df['Close'],
        name=f"{stock}",
    ),
    secondary_y=False,  # Assign to the primary y-axis
)

# Add the scatter plot for negative volatility
fig.add_trace(
    go.Scatter(
        x=stock_df.index,
        y=stock_df[f"LR"],
        mode='lines',
        name=f"Negative volatility",
        line=dict(color='#FF5252', width=2, dash='dot'),
    ),
    secondary_y=True,  # Assign to the secondary y-axis
)

# Update y-axes to use a logarithmic scale (optional)
fig.update_yaxes(type='log', secondary_y=False)  # Log scale for primary y-axis
fig.update_yaxes(title_text="Negative Volatility", secondary_y=True)

# Update layout
fig.update_layout(
    title='Standard candlesticks and trading signals based on EMAs',
    yaxis_title=f"{stock} Stock",
    xaxis_rangeslider_visible=False,
    width=1200,
    height=800,
)

# Show the plot
fig.show()

### Identify extrema: full ATR

In [ ]:
outliers = int(len(stock_df) * 0.05)

# Your original plot
ax = stock_df['TR'].plot(label='ATR', linestyle='-')

# Get the indices of the top 10 values in 'TR'
markers_on = stock_df['TR'].sort_values(ascending=False)[:outliers]

label = f'top {outliers} extrema'

# Loop over the selected indices and plot markers
for index in markers_on.index:
    ax.plot(index, stock_df['TR'].loc[index], 'oy', mfc='none', label=label)
    label = None

plt.legend()
plt.show()

### Identify extrema: negative range

In [ ]:
outliers = int(len(stock_df) * 0.05)

# Your original plot
ax = stock_df['LR'].plot(label='line', linestyle='-')

# Get the indices of the top 10 values in 'TR'
markers_series = stock_df['LR'].sort_values(ascending=False)[:outliers]

label = f'top {outliers} extrema'

# Loop over the selected indices and plot markers
for index in markers_series.index:
    ax.plot(index, stock_df['LR'].loc[index], 'oy', mfc='none', label=label)
    label = None

plt.legend()
plt.show()

### Identify extrema: positive range

In [ ]:
outliers = int(len(stock_df) * 0.05)

# Your original plot
ax = stock_df['HR'].plot(label='line', linestyle='-')

# Get the indices of the top 10 values in 'TR'
markers_series = stock_df['HR'].sort_values(ascending=False)[:outliers]

label = f'top {outliers} extrema'

# Loop over the selected indices and plot markers
for index in markers_series.index:
    ax.plot(index, stock_df['HR'].loc[index], 'oy', mfc='none', label=label)
    label = None

plt.legend()
plt.show()

### Extract isolated maxima from series

To get an estimate for the strongest daily movement in a turbulent trading phase

Note, that this extremum and neigboring suppressed extrema can accumulate to much larger numbers.

In [ ]:
def find_isolated_spikes(series, num_spikes=10, n_distance=5):
    """
    Find isolated maximum spikes in a Pandas series with a safety distance between each event.

    :param series: Input Pandas Series.
    :param num_spikes: Number of spikes to identify.
    :param n_distance: Safety distance between each identified spike.
    :return: DataFrame with the positions and values of the identified spikes.
    """

    # Copy the series to avoid modifying the original
    series_copy = series.copy()

    # List to hold the positions and values of the identified spikes
    spikes = []

    distance = n_distance * datetime.timedelta(days=1)

    max_seridx = max(series.index)
    min_seridx = min(series.index)

    for _ in range(num_spikes):
        # Find the index of the maximum value in the series
        max_idx = series_copy.idxmax()
        max_value = series_copy[max_idx]
        # print(f"{num_spikes}: {max_idx} {max_value}")

        # Save the maximum spike (index and value)
        spikes.append((max_idx, max_value))

        # Suppress the values around the maximum spike
        start_idx = max(min_seridx, max_idx - distance)
        end_idx = min(max_seridx, max_idx + distance)

        series_copy[start_idx:end_idx] = np.nan

    # Convert the result to a DataFrame for better readability
    spikes = list(zip(*spikes))
    series = pd.Series(spikes[1], index=spikes[0])

    return series


high_markers_series = find_isolated_spikes(stock_df['HR'], num_spikes=10, n_distance=5)
low_markers_series = find_isolated_spikes(stock_df['LR'], num_spikes=10, n_distance=5)

# Merge series to df
extrema_df = pd.concat(
    [high_markers_series, low_markers_series], axis=1, keys=['HR', 'LR']
)
extrema_df.sort_index(ascending=True, axis=0, inplace=True)

# Plot extrema
fig, axes = plt.subplots(nrows=2, ncols=1)

# Your original plot
ax_hr = stock_df['HR'].plot(ax=axes[0], label='HR line', linestyle='-')
ax_lr = stock_df['LR'].plot(ax=axes[1], label='LR line', linestyle='-')

label = f'top {outliers} iso-extrema'

# Loop over the selected indices and plot markers
for index in extrema_df[~extrema_df['HR'].isna()].index:
    ax_hr.plot(index, stock_df['HR'].loc[index], 'oy', mfc='none', label=label)
    label = None

label = f'top {outliers} iso-extrema'
for index in extrema_df[~extrema_df['LR'].isna()].index:
    ax_lr.plot(index, stock_df['LR'].loc[index], 'oy', mfc='none', label=label)
    label = None

fig.set_figwidth(12)
fig.set_figheight(8)
ax_hr.legend(loc="upper left")
ax_lr.legend(loc="upper left")

# plt.figure(figsize=(12,6))
# plt.legend()
plt.show()

del high_markers_series, low_markers_series

### Calculate different averages

In [ ]:
##---- Different means
scaler = 1.0

# Take valid extrema
for item in ['HR', 'LR']:
    markers_series = extrema_df[~extrema_df[item].isna()][item]

    arithmetic_mean = markers_series.sum() / len(markers_series) * scaler
    harmonic_mean = len(markers_series) / (1 / markers_series).sum() * scaler
    geometric_mean = np.exp(np.log(markers_series).mean()) * scaler

    print(
        f""" {item} extrema:
    {'Arithmetic mean' : <16}: {arithmetic_mean:8.2f}
    {'Harmonic mean' : <16}: {harmonic_mean:8.2f}
    {'Geometric mean' : <16}: {geometric_mean:8.2f}
    {'Extrema' : <16}: {extrema_df[item].nlargest(3).to_list()}
    """
    )